# Out-of-State-Contributions: Data Importation and Preparation

In [1]:
from functools import reduce
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

<IPython.core.display.Javascript object>

In [2]:
nimp_key = open("nimp_api_key.txt", "r").readline()

## Import and format the data

### Import and format contribution-level data from the [National Institute on Money in Politics](https://www.followthemoney.org/) for gubernatorial, attorney general, secretary of state, state supreme court, state senate and state house candidates in 2018, 2014 and 2010.

Download and save each cycle's contributions data and then concatenate them into a single file.

In [ ]:
%%bash
api_key=$(cat nimp_api_key.txt)
wget https://www.followthemoney.org/aaengine/aafetch.php\?dt\=1\&y\=2018\&c-exi\=1\&c-r-oc\=Z10,Z70\&c-r-ot\=G,S,H,J\&gro\=c-t-id,d-id\&APIKey\=$api_key\&mode\=csv -O data/raw/contributions_18.csv
wget https://www.followthemoney.org/aaengine/aafetch.php\?dt\=1\&y\=2014\&c-exi\=1\&c-r-oc\=Z10,Z70\&c-r-ot\=G,S,H,J\&gro\=c-t-id,d-id\&APIKey\=$api_key\&mode\=csv -O data/raw/contributions_14.csv
wget https://www.followthemoney.org/aaengine/aafetch.php\?dt\=1\&y\=2010\&c-exi\=1\&c-r-oc\=Z10,Z70\&c-r-ot\=G,S,H,J\&gro\=c-t-id,d-id\&APIKey\=$api_key\&mode\=csv -O data/raw/contributions_10.csv
head -1 "data/raw/contributions_18.csv" >> "data/raw/contributions.csv"
sed '1d' "data/raw/contributions_18.csv" >> "data/raw/contributions.csv"
sed '1d' "data/raw/contributions_14.csv" >> "data/raw/contributions.csv"
sed '1d' "data/raw/contributions_10.csv" >> "data/raw/contributions.csv"

Import the contributions data.

In [4]:
contributions = pd.read_csv("data/raw/contributions.csv", usecols=["Candidate", "Election_Jurisdiction", "Election_Year", "Office_Sought", "Contributor", "Amount", "Date", "In-State"], error_bad_lines=False)
contributions.columns = ["candidate", "state", "year", "office", "contributor", "amount", "date", "in_out_state"]
contributions.head(1)

/home/jyerardi/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,candidate,state,year,office,contributor,amount,date,in_out_state
0,"RAUNER, BRUCE VINCENT & SANGUINETTI, EVELYN PA...",IL,2018,GOVERNOR / LIEUTENANT GOVERNOR,"RAUNER, BRUCE VINCENT","50,000,000.00",2016-12-20,1.00


Delete the data.

In [7]:
%%bash
rm "data/raw/contributions_18.csv"
rm "data/raw/contributions_14.csv"
rm "data/raw/contributions_10.csv"
rm "data/raw/contributions.csv"

Our first task is to determine a data cut-off point for prior election cycles so we can make accurate comparisons across cycles.